In [2]:
import urllib.request
import bs4
import pandas

response = urllib.request.urlopen('https://vk.com/topic-22522055_25921627?offset=20')
soup = bs4.BeautifulSoup(response, 'html.parser')

In [3]:
nodes = soup.findAll('div', {'class': 'post_item'})
print(len(nodes))

20


In [40]:
for node in nodes:
    fromId = node.find('a', {'class': 'pi_author'}).get('href')[1:]
    toId = node.find('a', {'class': 'mem_link'})
    if (toId):
        toId = toId.get('href')[1:]
    date = node.find('a', {'class', 'item_date'})
    likes = node.find('a', {'class', 'item_like _i'})

In [41]:
monthRus = {
    'янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'
}

In [50]:
str1 = 'time'
str2 = 'datetime'

from pandas import pd

ImportError: cannot import name 'pd'

SyntaxError: invalid syntax (<string>, line 1)